# ML in Finance Group Project
### Group 2: Barbara Capl, Mathias Lüthi, Pamela Matias, Stefanie Rentsch
## 1. Preperation of Datasets
In this part of the code the data is imported from the Wharton data source. It's then cleaned up and put into usable attribute matrices for further feature selection.

In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np

### Market Data from Wharton
The financial data from the Wharton Database is loaded in. The redundant identification columns and columns which are more than 2/3 empty values are dropped.  

In [12]:
# Load in data from the Wharton Database
wharton = pd.read_csv('Data/Whartondata.csv', sep=',', header=0)

# Delete unusable columns
wharton = wharton.drop(['NAMEENDT','SHRCD','EXCHCD','SICCD','TICKER','COMNAM', 'NCUSIP', 'TSYMBOL',
              'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD'], 1)
wharton = wharton.drop(['DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'HSICMG', 'HSICIG', 'DIVAMT',
              'SHRCLS', 'ACPERM', 'ACCOMP', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'NMSIND',
              'MMCNT', 'NSDINX', 'DCLRDT', 'PAYDT', 'RCRDDT', 'DISTCD', 'FACPR', 'FACSHR',
              'TRTSCD', ], axis=1)

# Rename PERMNO column
wharton.columns.values[0] = 'permno'

# Format date into string with only year and month
wharton['date'] = wharton.date.astype(str).str[:4] + '-' + wharton.date.astype(str).str[4:6]


# Calculate SPREAD manually
wharton['SPREAD'] = wharton['BID'] - wharton['ASK']

####### print(wharton.isnull().sum())


display(wharton.head())

,permno,date,NAICS,PRIMEXCH,TRDSTAT,SECSTAT,CUSIP,SHRFLG,SHRENDDT,BIDLO,...,CFACSHR,ALTPRC,SPREAD,ALTPRCDT,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10107,2006-01,511210.0,Q,A,R,59491810,0.0,20060330.0,26.28,...,1.0,28.15,0.02,20060131.0,0.076482,0.040044,0.038938,0.076354,0.075150,0.025467
1,10107,2006-02,511210.0,Q,A,R,59491810,0.0,NaN,26.39,...,1.0,26.87,-0.01,20060228.0,-0.045471,-0.001637,-0.003552,0.004836,0.003476,0.000453
2,10107,2006-03,511210.0,Q,A,R,59491810,0.0,20060423.0,26.85,...,1.0,27.21,0.00,20060331.0,0.012653,0.019053,0.017585,0.036978,0.035232,0.011065
3,10107,2006-04,511210.0,Q,A,R,59491810,0.0,20060629.0,24.15,...,1.0,24.15,0.00,20060428.0,-0.112459,0.012965,0.011494,0.009791,0.008497,0.012187
4,10107,2006-05,511210.0,Q,A,R,59491810,0.0,NaN,22.56,...,1.0,22.65,0.00,20060531.0,-0.062112,-0.031045,-0.033025,-0.044331,-0.045982,-0.030917


### Financial Ratios from Wharton
The financial ratios from the Wharton Database (received on OLAT) are loaded in. The adate and qdate are dropped because they are not relevant. Formatting "divyeld" from percentage to float.

In [11]:
# Load in the financial ratios from OLAT
ratios = pd.read_csv('Data/Ratios.csv', sep=',', header=0)

# Delete unusable columns
ratios = ratios.drop(['adate', 'qdate'], axis=1)

ratios.columns.values[1] = 'date'
ratios['date'] = ratios.date.str[6:] + '-' + ratios.date.str[3:5]

# Remove percentages in row "divyield" and divide with 100 (so its decimal percentage) with string split
ratios['divyield'] = ratios['divyield'].str.rstrip('%').astype('float')/100

####### print(ratios.isnull().sum())


display(ratios.head())

,permno,date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10107,2006-01,33.861,0.176,15.245,23.074,23.264,23.856,23.856,7.211,...,1.296,0.151,0.025,0.0,0.055,6.019,11.538,0.0114,15.506,2.109
1,10107,2006-02,31.756,0.163,14.951,21.496,21.846,22.392,22.392,6.713,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,0.0134,14.555,1.838
2,10107,2006-03,31.820,0.163,14.951,21.768,22.122,22.675,22.675,6.727,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,0.0132,14.739,1.842
3,10107,2006-04,28.176,0.163,14.951,19.320,19.634,20.125,20.125,5.957,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,0.0149,13.081,1.666
4,10107,2006-05,25.921,0.151,15.120,17.695,17.835,17.976,17.976,5.419,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,0.0159,-5.842,1.480


### Merging the Dataset

In [13]:
# Merging the two dataframes on date and permno
data = pd.merge(wharton, ratios, left_on=['date', 'permno'], right_on=['date', 'permno'])
display(data.head())

,permno,date,NAICS,PRIMEXCH,TRDSTAT,SECSTAT,CUSIP,SHRFLG,SHRENDDT,BIDLO,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,PEG_1yrforward,PEG_ltgforward
0,10107,2006-01,511210.0,Q,A,R,59491810,0.0,20060330.0,26.28,...,1.296,0.151,0.025,0.0,0.055,6.019,11.538,0.0114,15.506,2.109
1,10107,2006-02,511210.0,Q,A,R,59491810,0.0,NaN,26.39,...,1.323,0.151,0.025,0.0,0.036,6.281,10.280,0.0134,14.555,1.838
2,10107,2006-03,511210.0,Q,A,R,59491810,0.0,20060423.0,26.85,...,1.323,0.151,0.025,0.0,0.036,6.293,10.410,0.0132,14.739,1.842
3,10107,2006-04,511210.0,Q,A,R,59491810,0.0,20060629.0,24.15,...,1.323,0.151,0.025,0.0,0.036,5.573,9.239,0.0149,13.081,1.666
4,10107,2006-05,511210.0,Q,A,R,59491810,0.0,NaN,22.56,...,1.388,0.150,0.025,0.0,0.024,5.496,0.709,0.0159,-5.842,1.480


### Adding additonal Features

In [5]:
# Adding reporting period as seasonality attribute

# Breakdown of categorical data


### Creating of Attribute Matrices and Response Vectors

In [14]:
# Creating a responce vector and an attribute matrix
forcast_periods = [1, 3, 6, 12]
for i in forcast_periods:
    data['return_' + str(i)] = np.where((data['permno'] == data['permno'].shift(i)), data['PRC'] / data['PRC'].shift(i), None)

# Creating a responce vector and an attribute matrix (for different forecast periods can be done later)
data_1 = data.dropna(subset=['return_1'])
response_1 = np.where(data_1.return_1 >= 1, 1, 0)
attributes_ratios_1 = data_1.iloc[:, 28:-4]
attributes_additional_1 = data_1.iloc[:, 2:-4]